Problem 1: The Multi-Protocol Echo & Time Station

Concepts: TCP Servers (11.2), UDP Servers (11.3), and IP Address Management (11.4).

You want to build a server that can handle two types of "pings." A TCP Echo service for reliable message testing and a UDP Time service for quick, lightweight time checks. You also need a utility to verify if an incoming client IP belongs to your local office network.

Task:

Implement a TCPServer that receives a message and sends it back to the client.

Implement a UDPServer that ignores the incoming message and simply returns the current server time.

Use the ipaddress module to create a function is_authorized(ip_str) that returns True if the IP is within the subnet 192.168.1.0/24.

In [1]:
import ipaddress
import time
from socketserver import BaseRequestHandler, TCPServer, UDPServer
import threading

class EchoHandler(BaseRequestHandler):
    def handle(self):
        data = self.request.recv(1024)
        if data:
            self.request.send(data)

class TimeHandler(BaseRequestHandler):
    def handle(self):
        msg, skt = self.request
        resp = time.ctime().encode('ascii')
        skt.sendto(resp, self.client_address)

def is_authd(ip_str):
    net = ipaddress.ip_network('192.168.1.0/24')
    return ipaddress.ip_address(ip_str) in net


def start_tcp():
    with TCPServer(("", 20000), EchoHandler) as srv:
        print("TCP Echo server on port 20000")
        srv.serve_forever()


def start_udp():
    with UDPServer(("", 20001), TimeHandler) as srv:
        print("UDP Time server on port 20001")
        srv.serve_forever()


print("Auth check:", is_authd("192.168.1.10"))

threading.Thread(target=start_tcp, daemon=True).start()
threading.Thread(target=start_udp, daemon=True).start()

input("Servers running. Press Enter to stop...\n")

Auth check: True
TCP Echo server on port 20000
UDP Time server on port 20001


'dsfsdaf'

In [ ]:
from wsgiref.simple_server import make_server

from urllib import request

def hello_and_fetch(environ, start_response):
    with request.urlopen('http://httpbin.org/get') as resp:
        ext_data = resp.read().decode('utf-8')

    start_response('200 OK', [('Content-type', 'text/html')])

    content = f"<h1>Hello!</h1><p>External Status: {ext_data[:50]}...</p>"

    return [content.encode('utf-8')]


httpd = make_server('', 8080, hello_and_fetch)
print('we are live on port 8080...')
httpd.serve_forever()

In [3]:
from xmlrpc.server import SimpleXMLRPCServer

class MathServer:
    def add(self, x,y):
        return x + y
    def div(self, x,y):
        return x/0

serv = SimpleXMLRPCServer(('localhost', 15000))
serv.register_instance(MathServer())
serv.serve_forever()

127.0.0.1 - - [30/Dec/2025 12:07:24] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2025 12:07:24] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2025 12:07:53] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 

In [ ]:
from xmlrpc.client import ServerProxy

s = ServerProxy('http://localhost:15000')

print(s.add(22,2))
print(s.div(32,3))

In [ ]:
from _socket import socket
import socket, ssl, hmac, os, multiprocessing

from multiprocessing.reduction import send_handle, recv_handle

SECRKEY = b'haseeb234'

def authenticate(conn):
    nonce = os.urandom(32)
    conn.sendall(nonce)
    digest = hmac.new(SECRKEY, nonce, 'sha256').digest()
    client_digest = conn.recv(len(digest))
    return hmac.compare_digest(digest, client_digest)


def worker(pipe):
    while True:
        fd = recv_handle(pipe)
        with socket.fromfd(fd, socket.AF_INET, socket.SOCK_STREAM) as s:
            s.sendall(b"Access granted to vault!...")

def gatekeeper(adress, pipe, worker_pid):
    raw_s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    raw_s.bind(adress)
    raw_s.listen(5)

    while True:
        client, adr = raw_s.accept()
        if authenticate(client):
            send_handle(pipe, client.fileno(), worker_pid)
        client.close()
